In [ ]:
# Sentiment vs Trader Performance

In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

In [ ]:
# Plotting defaults
plt.rcParams['figure.figsize'] = (8,4)

In [3]:
SENT_FILE = 'fear_greed_index.csv' 
TRADES_FILE = 'historical_data.csv'
OUT_DIR = Path('analysis_outputs')
OUT_DIR.mkdir(exist_ok=True)

In [5]:
def safe_read(path):
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
    
    # Try common formats
    if path.suffix.lower() in ['.csv', '.txt']:
        return pd.read_csv(path, low_memory=False)
    if path.suffix.lower() in ['.parquet', '.pq']:
        return pd.read_parquet(path)
    
    # Fallback
    return pd.read_csv(path, low_memory=False)

In [8]:
def find_column_like(df, keywords):
    keys = [k.lower() for k in keywords]
    for c in df.columns:
        low = c.lower()
        for k in keys:
            if k in low:
                return c
    return None

In [9]:
sent = safe_read(SENT_FILE)
trades = safe_read(TRADES_FILE)


print('Sentiment rows, cols:', sent.shape)
print('Trades rows, cols:', trades.shape)

Sentiment rows, cols: (2644, 4)
Trades rows, cols: (211224, 16)


In [10]:
print('\nSentiment head:')
print(sent.head())
print('\nTrades head:')
print(trades.head())


Sentiment head:
    timestamp  value classification        date
0  1517463000     30           Fear  2018-02-01
1  1517549400     15   Extreme Fear  2018-02-02
2  1517635800     40           Fear  2018-02-03
3  1517722200     24   Extreme Fear  2018-02-04
4  1517808600     11   Extreme Fear  2018-02-05

Trades head:
                                      Account  Coin  Execution Price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   

   Size Tokens  Size USD Side     Timestamp IST  Start Position Direction  \
0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
2       144.09   